## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [12]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [13]:
# load data

players = pd.read_csv("../data/processed-data/0_selection_players.csv", index_col=0)
n_players = players.shape[0]
market_values = pd.read_csv("../data/processed-data/0_selection_mv.csv", index_col=0,).drop('Player', axis=1).values
n_time = market_values.shape[1]
n_trials = int(market_values.shape[0]/n_players)
market_values = market_values.reshape((n_trials, n_players, n_time))
salaries = np.mean(market_values, axis = 0) * 0.1

# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [14]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 40
starting_budget = 1500000
wage_rev = 0.63
rho = 0.05
ret = 0.02

time = np.arange(n_time)
trials = np.arange(n_trials)

In [15]:
# Decision variables
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")
lent_out = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "lentout_%s_%s")
borrowed = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "borrowed_%s_%s")

budget = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Budget_%s_%s", lb=0)
team_value = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Team Value_%s_%s")

sunset_value = ftcp.continuous_var_list(keys=trials, name = "Sunset Value_%s")
optim_value = ftcp.continuous_var(name = "Optimisation Value")

In [16]:
# Non decision variables
# Total number of players owned by the team is not the same as number of players playing in tournament
N_owned = ftcp.integer_var_list(keys = time, name = 'Number of Players owned by the team_%s', ub=30)
N_team = ftcp.integer_var_list(keys = time, name = 'Number of Players in the team_%s', lb=22, ub=25)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)       

In [17]:
# Set up Constraints for number of players
for year in time:
    for trial in trials:
        ###
        # Not used in optim, but will be used as constraint to control costs
        ###
        previous_budget = starting_budget if year == 0 else budget[year-1, trial]
        ftcp.add_constraint(budget[year, trial] == previous_budget * (1 + ret)
                            + (1/wage_rev - 1) * ftcp.sum((assigned[index, year] + 1.1*borrowed[index, year] - 0.1*lent_out[index, year]) * salaries[index, year] for index in range(n_players)))
        
        ftcp.add_constraint(team_value[year, trial] == ftcp.sum((assigned[index, year] + borrowed[index, year]) * market_values[trial, index, year] for index in range(n_players))/((1 + rho)**year)
                            - ftcp.sum((assigned[index, year] + 1.1*borrowed[index, year] - 0.1*lent_out[index, year]) * salaries[index, year] for index in range(n_players)))
    
    ftcp.add_constraint(N_gk[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['FW'] for index, player in players.iterrows()))
    
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_team[year])
    ftcp.add_constraint(N_owned[year] == ftcp.sum(assigned[index, year] for index, player in players.iterrows()))
    
    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year] * (player['Age'] + year) <= max_age)
        
        ftcp.add_constraint(assigned[index, year] <= (player['Raritan']))
        ftcp.add_constraint(lent_out[index, year] <= assigned[index, year])
        ftcp.add_constraint(borrowed[index, year] <= (1 - player['Raritan']))
        
for trial in trials:        
    ftcp.add_constraint(sunset_value[trial] == ftcp.sum((assigned[index, time[-1]] + borrowed[index, time[-1]]) * market_values[trial, index, time[-1]] for index in range(n_players))/((1 + rho)**(n_time)))
    
ftcp.add_constraint(optim_value == (ftcp.sum(team_value[x, y] for x in time for y in trials) + ftcp.sum(sunset_value[x] for x in trials))/n_trials)

docplex.mp.LinearConstraint[](Optimisation Value,EQ,0.333Team Value_0_0+0.333Team Value_0_1+0.333Team Value_0_2+0.333Team Value_1_0+0.333Team Value_1_1+0.333Team Value_1_2+0.333Team Value_2_0+0.333Team Value_2_1+0.333Team Value_2_2+0.333Team Value_3_0+0.333Team Value_3_1+0.333Team Value_3_2+0.333Team Value_4_0+0.333Team Value_4_1+0.333Team Value_4_2+0.333Team Value_5_0+0.333Team Value_5_1+0.333Team Value_5_2+0.333Team Value_6_0+0.333Team Value_6_1+0.333Team Value_6_2+0.333Team Value_7_0+0.333Team Value_7_1+0.333Team Value_7_2+0.333Team Value_8_0+0.333Team Value_8_1+0.333Team Value_8_2+0.333Team Value_9_0+0.333Team Value_9_1+0.333Team Value_9_2+0.333Sunset Value_0+0.333Sunset Value_1+0.333Sunset Value_2)

In [18]:
ftcp.maximize(optim_value)
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 84784
   - binary=84660, integer=60, continuous=64
 - number of constraints: 113004
   - linear=113004
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [19]:
ftcp.solve(log_output=True, clean_before_solve=True)
ftcp.solve_status

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 90543 rows and 34633 columns.
Reduced MIP has 22461 rows, 50151 columns, and 161363 nonzeros.
Reduced MIP has 50080 binaries, 65 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.41 sec. (388.18 ticks)
Found incumbent of value 5.3794685e+09 after 1.75 sec. (577.52 ticks)
Probing time = 0.04 sec. (12.22 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 4 rows and 4 columns.
Reduced MIP has 22457 rows, 50147 columns, and 140917 nonzeros.
Reduced MIP has 50080 binaries, 65 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.37 sec. (106.70 ticks)
Probing time = 0.03 sec. (9.86 ticks)
Clique table members: 22395.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 1.45 sec. (666.93 ticks

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [20]:
ftcp.print_solution()


objective: 16149112906.863
  "assign_91_0"=1
  "assign_91_1"=1
  "assign_91_2"=1
  "assign_91_3"=1
  "assign_91_4"=1
  "assign_91_5"=1
  "assign_91_6"=1
  "assign_91_7"=1
  "assign_91_8"=1
  "assign_91_9"=1
  "assign_224_0"=1
  "assign_224_1"=1
  "assign_224_2"=1
  "assign_224_3"=1
  "assign_224_4"=1
  "assign_224_5"=1
  "assign_224_6"=1
  "assign_224_7"=1
  "assign_224_8"=1
  "assign_224_9"=1
  "assign_283_0"=1
  "assign_283_1"=1
  "assign_283_2"=1
  "assign_283_3"=1
  "assign_283_4"=1
  "assign_283_5"=1
  "assign_283_6"=1
  "assign_283_7"=1
  "assign_283_8"=1
  "assign_416_9"=1
  "assign_456_8"=1
  "assign_456_9"=1
  "assign_501_0"=1
  "assign_501_1"=1
  "assign_501_2"=1
  "assign_501_3"=1
  "assign_501_4"=1
  "assign_501_5"=1
  "assign_501_6"=1
  "assign_501_7"=1
  "assign_501_8"=1
  "assign_571_0"=1
  "assign_571_1"=1
  "assign_571_2"=1
  "assign_571_3"=1
  "assign_571_4"=1
  "assign_571_5"=1
  "assign_571_6"=1
  "assign_571_7"=1
  "assign_571_8"=1
  "assign_571_9"=1
  "assign_616_

In [21]:
ftcp.solution.export("optim_solution.json")

In [22]:
# import docplex.mp.conflict_refiner as cr

# ftcp.solve()
# solve_status = ftcp.get_solve_status()
# if solve_status.name in ['INFEASIBLE_SOLUTION', 'INFEASIBLE_OR_UNBOUNDED_SOLUTION']:
#     cref = cr.ConflictRefiner()
#     print('show some of the constraints that can be removed to arrive at a minimal conflict')
#     cref_result = cref.refine_conflict(ftcp, display=True)  # display flag is to show the conflicts

#     cref_result.display()
#     cref_result.as_output_table()